In [ ]:
!nvidia-smi

Mon Jun  9 16:46:17 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   57C    P8             13W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Data Ingestion

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("emineyetm/fake-news-detection-datasets")

print("Path to dataset files:", path)

100%|██████████| 41.0M/41.0M [00:01<00:00, 33.4MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/emineyetm/fake-news-detection-datasets/versions/1


In [ ]:
import os
os.listdir(f"{path}/News _dataset")

['Fake.csv', 'True.csv']

In [ ]:
import pandas as pd
true = pd.read_csv(os.path.join(path,"News _dataset/True.csv"))
fake = pd.read_csv(os.path.join(path,"News _dataset/Fake.csv"))
true['label']=1
fake['label']=0

In [ ]:
df = pd.concat([true,fake],ignore_index=True)

In [ ]:
df = df.sample(frac=1,random_state=42).reset_index(drop=True)

In [ ]:
df

,title,text,subject,date,label
0,BREAKING: GOP Chairman Grassley Has Had Enoug...,"Donald Trump s White House is in chaos, and th...",News,"July 21, 2017",0
1,Failed GOP Candidates Remembered In Hilarious...,Now that Donald Trump is the presumptive GOP n...,News,"May 7, 2016",0
2,Mike Pence’s New DC Neighbors Are HILARIOUSLY...,Mike Pence is a huge homophobe. He supports ex...,News,"December 3, 2016",0
3,California AG pledges to defend birth control ...,SAN FRANCISCO (Reuters) - California Attorney ...,politicsNews,"October 6, 2017",1
4,AZ RANCHERS Living On US-Mexico Border Destroy...,Twisted reasoning is all that comes from Pelos...,politics,"Apr 25, 2017",0
...,...,...,...,...,...
44893,Nigeria says U.S. agrees delayed $593 million ...,ABUJA (Reuters) - The United States has formal...,worldnews,"December 27, 2017",1
44894,Boiler Room #62 – Fatal Illusions,Tune in to the Alternate Current Radio Network...,Middle-east,"June 29, 2016",0
44895,ATHEISTS SUE GOVERNOR OF TEXAS Over Display on...,I m convinced the Freedom From Religion group...,Government News,"Feb 27, 2016",0
44896,Republican tax plan would deal financial hit t...,WASHINGTON (Reuters) - The Republican tax plan...,politicsNews,"November 2, 2017",1


In [ ]:
df['subject'].unique()

array(['News', 'politicsNews', 'politics', 'worldnews', 'Government News',
       'left-news', 'Middle-east', 'US_News'], dtype=object)

In [ ]:
er=[]
for i in range(0,len(df)):
  rs = len(df['text'][i])
  er.append(rs)
print(f"Avg length of text = {sum(er)/len(df)}")

Avg length of text = 2469.1096930820972


## Data Transformation

In [ ]:
df.columns

Index(['title', 'text', 'subject', 'date', 'label'], dtype='object')

In [ ]:
!pip install text_prettifier

In [ ]:
from text_prettifier import TextPrettifier

# Initialize the TextPrettifier
prettifier = TextPrettifier()
def text_cleaner(text):
    text = text.lower()
    text = prettifier.remove_contractions(text)
    text = prettifier.remove_emojis(text)
    text = prettifier.remove_html_tags(text)
    text = prettifier.remove_urls(text)
    # keep stopwords and punctuation for BERT or LSTM
    return text

In [ ]:
df['text']=df['text'].apply(text_cleaner)

In [ ]:
df['text'][0]

'donald trump s white house is in chaos, and they are trying to cover it up. their russia problems are mounting by the hour, and they refuse to acknowledge that there are problems surrounding all of this. to them, it s fake news, or a hoax. however, the facts bear things out differently, and it seems that there are now cracks in the congressional public leadership.chuck grassley (r-iowa), who heads the senate judiciary committee, is fed up. he is now demanding that donald trump, jr. and former 2016 trump campaign manager paul manafort testify before his committee regarding the now infamous shady meeting between donald trump and the shady russian lawyer who promised dirt on 2016 democratic presidential nominee hillary clinton. in fact, this information is due, well, now. this demand sends a few signals to team trump most notably that they should not fire special counsel robert mueller under any circumstances, despite the fact that it seems that this seems to be what trump s white house 

## Model finetuning

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained("therealcyberlord/fake-news-classification-distilbert")
model = AutoModelForSequenceClassification.from_pretrained("therealcyberlord/fake-news-classification-distilbert")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd

# Split the dataset (80% train, 20% test)
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)

In [ ]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [ ]:
def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=512)

train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)


Map:   0%|          | 0/35918 [00:00<?, ? examples/s]

Map:   0%|          | 0/8980 [00:00<?, ? examples/s]

In [ ]:
from transformers import Trainer, TrainingArguments
args = TrainingArguments(
    output_dir="./fine_tuned_roberta",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
     report_to="none",
    num_train_epochs=2,
    logging_steps=50,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [ ]:
train_dataset

Dataset({
    features: ['title', 'text', 'subject', 'date', 'label', '__index_level_0__', 'input_ids', 'attention_mask'],
    num_rows: 35918
})

In [ ]:
trainer.train()

Step,Training Loss
50,0.067400
100,0.016900
150,0.046100
200,0.018700
250,0.036600
300,0.023500
350,0.000100
400,0.000100
450,0.000000
500,0.000000


TrainOutput(global_step=8980, training_loss=0.004032443900954106, metrics={'train_runtime': 4204.8386, 'train_samples_per_second': 17.084, 'train_steps_per_second': 2.136, 'total_flos': 9515928049852416.0, 'train_loss': 0.004032443900954106, 'epoch': 2.0})

In [ ]:
model.save_pretrained("distilbert_fake_news_model")
tokenizer.save_pretrained("distilbert_fake_news_tokenizer")

('distilbert_fake_news_tokenizer/tokenizer_config.json',
 'distilbert_fake_news_tokenizer/special_tokens_map.json',
 'distilbert_fake_news_tokenizer/vocab.txt',
 'distilbert_fake_news_tokenizer/added_tokens.json',
 'distilbert_fake_news_tokenizer/tokenizer.json')

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

# Load the saved model
model_path = "/content/distilbert_fake_news_model"
tokenizer_path='/content/distilbert_fake_news_tokenizer'
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)
model.eval()

# Move to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Sample test text
sample_text = "India successfully lands Chandrayaan-3 near the Moon's south pole"

# Tokenize
inputs = tokenizer(sample_text, return_tensors="pt", truncation=True, padding=True).to(device)

# Predict
with torch.no_grad():
    outputs = model(**inputs)
    probs = F.softmax(outputs.logits, dim=-1)
    prediction = torch.argmax(probs, dim=-1).item()

# Output
label = "Real News" if prediction == 1 else "Fake News"
print(f"📰 Text: {sample_text}")
print(f"📢 Prediction: {label} ({probs[0][prediction].item():.4f} confidence)")


📰 Text: India successfully lands Chandrayaan-3 near the Moon's south pole
📢 Prediction: Real News (1.0000 confidence)
